In [10]:
import openclean
import glob
import pandas as pd
import pyspark 
import numpy as np

In [58]:
pyspark.__version__

'3.2.0'

In [57]:
np.__version__

'1.21.3'

In [56]:
pd.__version__

'1.3.4'

In [7]:
import glob

In [8]:
glob.glob("*")

['DOB_Job_Application_Filings.csv',
 'DOB_Job_Cleaning.ipynb',
 'ic3t-wcy2.tsv.gz',
 'README.md']

In [28]:
df  = pd.read_csv('./DOB_Job_Application_Filings.csv', dtype='object')

In [30]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1774833 entries, 0 to 1774832
Data columns (total 96 columns):
 #   Column                         Non-Null Count    Dtype 
---  ------                         --------------    ----- 
 0   Job #                          1774833 non-null  object
 1   Doc #                          1774833 non-null  object
 2   Borough                        1774833 non-null  object
 3   House #                        1774827 non-null  object
 4   Street Name                    1774827 non-null  object
 5   Block                          1774081 non-null  object
 6   Lot                            1774077 non-null  object
 7   Bin #                          1774833 non-null  object
 8   Job Type                       1774833 non-null  object
 9   Job Status                     1774833 non-null  object
 10  Job Status Descrp              1774833 non-null  object
 11  Latest Action Date             1774833 non-null  object
 12  Building Type               

In [47]:
df.count()

Job #                   1774833
Doc #                   1774833
Borough                 1774833
House #                 1774827
Street Name             1774827
                         ...   
GIS_LONGITUDE           1769176
GIS_COUNCIL_DISTRICT    1769176
GIS_CENSUS_TRACT        1769176
GIS_NTA_NAME            1769176
GIS_BIN                 1752710
Length: 96, dtype: int64

### if any rows are complete duplicates, drop them

In [48]:
df = df.drop_duplicates()

In [49]:
df

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
0,440673497,01,QUEENS,215-02,93 AVENUE,10618,00028,4226672,A2,R,...,N,NaN,A1,0,40.720067,-73.742696,23,542,Queens Village,4226672
1,440673512,01,QUEENS,10040,222 STREET,10780,00019,4231025,A2,R,...,N,NaN,A1,0,40.714365,-73.731171,27,568,Queens Village,4231025
2,421133972,02,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
3,421133972,03,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
4,210182309,02,BRONX,97,WEST 169 STREET,02519,00027,2130622,NB,P,...,N,NaN,V0,0,40.839538,-73.923177,16,211,Highbridge,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774828,302042858,01,BROOKLYN,2770,OCEAN AVENUE,07404,00020,3202124,A3,X,...,N,NaN,D1,0,40.593831,-73.950368,48,592,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,3202124
1774829,302042867,01,BROOKLYN,313,76 STREET,05940,00078,3148750,A2,R,...,N,NaN,B1,0,40.630611,-74.027435,43,66,Bay Ridge,3148750
1774830,302042876,01,BROOKLYN,1658,66 STREET,05559,00033,3134197,A2,X,...,N,NaN,C3,0,40.620486,-73.994067,38,252,Bensonhurst West,3134197
1774831,302042885,01,BROOKLYN,173,LEXINGTON AVENUE,01968,00043,3056361,A2,Q,...,N,NaN,F9,0,40.687159,-73.955516,36,233,Bedford,3056361


In [34]:
df.columns#[5,6,13,14,20,21,32,38,59,60,68,74,75,76,77,79]

Index(['Job #', 'Doc #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot',
       'Bin #', 'Job Type', 'Job Status', 'Job Status Descrp',
       'Latest Action Date', 'Building Type', 'Community - Board', 'Cluster',
       'Landmarked', 'Adult Estab', 'Loft Board', 'City Owned', 'Little e',
       'PC Filed', 'eFiling Filed', 'Plumbing', 'Mechanical', 'Boiler',
       'Fuel Burning', 'Fuel Storage', 'Standpipe', 'Sprinkler', 'Fire Alarm',
       'Equipment', 'Fire Suppression', 'Curb Cut', 'Other',
       'Other Description', 'Applicant's First Name', 'Applicant's Last Name',
       'Applicant Professional Title', 'Applicant License #',
       'Professional Cert', 'Pre- Filing Date', 'Paid', 'Fully Paid',
       'Assigned', 'Approved', 'Fully Permitted', 'Initial Cost',
       'Total Est. Fee', 'Fee Status', 'Existing Zoning Sqft',
       'Proposed Zoning Sqft', 'Horizontal Enlrgmt', 'Vertical Enlrgmt',
       'Enlargement SQ Footage', 'Street Frontage', 'ExistingNo. of Stories',
  

In [35]:
df.head()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
0,440673497,01,QUEENS,215-02,93 AVENUE,10618,00028,4226672,A2,R,...,N,NaN,A1,0,40.720067,-73.742696,23,542,Queens Village,4226672
1,440673512,01,QUEENS,10040,222 STREET,10780,00019,4231025,A2,R,...,N,NaN,A1,0,40.714365,-73.731171,27,568,Queens Village,4231025
2,421133972,02,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
3,421133972,03,QUEENS,88-36,139TH STREET,09620,00033,4623670,NB,P,...,N,NaN,G7,0,40.703428,-73.813796,24,214,Briarwood-Jamaica Hills,NaN
4,210182309,02,BRONX,97,WEST 169 STREET,02519,00027,2130622,NB,P,...,N,NaN,V0,0,40.839538,-73.923177,16,211,Highbridge,NaN


## Take an initial look at Job #

In [36]:
df['Job #'].value_counts(dropna=False)

103408705    20
122705000    13
120438240    13
320843110    12
121189828    11
             ..
121007775     1
121010235     1
320447557     1
121182399     1
302042894     1
Name: Job #, Length: 1584287, dtype: int64

In [37]:
df['Job #'].min()

'100321046'

In [38]:
df['Job #'].max()

'577777776'

In [39]:
df.loc[df['Job #'].str.startswith('0')]

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN


In [45]:
df['Job #'].count()

1774833

In [46]:
df['Job #'].nunique()

1584287

In [43]:
df.loc[df['Job #']=='103408705']

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
44792,103408705,02,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,R,...,A,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
44793,103408705,03,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,R,...,A,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76194,103408705,08,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76198,103408705,09,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76277,103408705,04,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76281,103408705,05,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76283,103408705,06,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76285,103408705,12,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76289,103408705,13,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345
76294,103408705,14,MANHATTAN,555,W 23 STREET,00695,07503,1012345,NB,P,...,NaN,NaN,R0,0,40.748400,-74.005836,3,99,Hudson Yards-Chelsea-Flatiron-Union Square,1012345


In [50]:
df.describe()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
count,1774833,1774833,1774833,1774827,1774827,1774081,1774077,1774833,1774833,1774833,...,1754629,172179,1768356,1774833,1769176,1769176,1769176,1769176,1769176,1752710
unique,1584287,20,5,35644,29552,13816,1971,363769,9,17,...,4,6373,199,13,217559,228333,51,1328,194,342789
top,103408705,01,MANHATTAN,1,BROADWAY,00016,00001,1015862,A2,X,...,N,10/24/2012,R0,0,40.748276,-73.984690,4,7,Midtown-Midtown South,3000000
freq,20,1583861,775334,17359,52273,6808,216446,2413,1117073,1050347,...,1519403,5513,209462,1772918,2409,2413,236660,16439,153870,15627


In [85]:
df.columns[:20]

Index(['Job #', 'Doc #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot',
       'Bin #', 'Job Type', 'Job Status', 'Job Status Descrp',
       'Latest Action Date', 'Building Type', 'Community - Board', 'Cluster',
       'Landmarked', 'Adult Estab', 'Loft Board', 'City Owned', 'Little e'],
      dtype='object')

## Describe columns in groups so they fit on screen

In [86]:
df[df.columns[:20]].describe()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,Job Status Descrp,Latest Action Date,Building Type,Community - Board,Cluster,Landmarked,Adult Estab,Loft Board,City Owned,Little e
count,1774833,1774833,1774833,1774827,1774827,1774081,1774077,1774833,1774833,1774833,1774833,1774833,1774833,1773538,1131340,1670714,1581393,1400794,149391,1046417
unique,1584287,20,5,35644,29552,13816,1971,363769,9,17,17,7373,2,106,2,4,2,2,4,5
top,103408705,01,MANHATTAN,1,BROADWAY,00016,00001,1015862,A2,X,SIGNED OFF,10/13/2017,OTHERS,105,N,N,N,N,Y,N
freq,20,1583861,775334,17359,52273,6808,216446,2413,1117073,1050347,1050347,793,1389719,210722,1122869,1461463,1581220,1393358,149388,989635


In [91]:
# Notes:
# Building Type looks binary and has 2 values + maybe NAN
# Cluster looks binary and has 2 values + maybe NAN
# Landmarked looks binary and has 4 values + maybe NAN
# Adult Estab looks binary and has 2 values + maybe NAN
# Loft Board looks binary and has 2 values + maybe NAN
# City Owned looks binary and has 4 values + maybe NAN
# Little e looks binary and has 5 values + maybe NAN


In [87]:
df[df.columns[20:40]].describe()

,PC Filed,eFiling Filed,Plumbing,Mechanical,Boiler,Fuel Burning,Fuel Storage,Standpipe,Sprinkler,Fire Alarm,Equipment,Fire Suppression,Curb Cut,Other,Other Description,Applicant's First Name,Applicant's Last Name,Applicant Professional Title,Applicant License #,Professional Cert
count,602206,1081480,555712,284297,49960,21452,14255,14386,116386,87703,306812,43370,61213,1041742,1039940,1774774,1774821,1774750,1725666,1396638
unique,2,2,1,1,1,1,2,1,1,1,1,1,1,1,16626,17353,35220,913,27087,5
top,Y,Y,X,X,X,X,X,X,X,X,X,X,X,X,GEN. CONSTR.,MICHAEL,HOQUE,PE,083278,Y
freq,384720,1081478,555712,284297,49960,21452,14254,14386,116386,87703,306812,43370,61213,1041742,316603,51058,19159,924243,19195,859575


In [92]:
# PC Filed -- Other all look binary, and have 1-2 values + maybe NAN
#
# Take a look at Other Description for weird strings

#  Lots of the same First and Last name

# Check names and titles

# APPLICATNT LICENCSE # NEEDS TO BE A STRING TO PERSERVE THE 0 ON IT (PROBABLY)

# Professional Cert looks binary and has 5 values + maybe NAN



In [88]:
df[df.columns[40:60]].describe()

,Pre- Filing Date,Paid,Fully Paid,Assigned,Approved,Fully Permitted,Initial Cost,Total Est. Fee,Fee Status,Existing Zoning Sqft,Proposed Zoning Sqft,Horizontal Enlrgmt,Vertical Enlrgmt,Enlargement SQ Footage,Street Frontage,ExistingNo. of Stories,Proposed No. of Stories,Existing Height,Proposed Height,Existing Dwelling Units
count,1774833,1765764,1768254,1195823,1444933,1309680,1774833,1774833,1774833,1774833,1774833,45822,32149,1774833,1774833,1774833,1774833,1774833,1774833,644092
unique,6622,6641,6708,6781,6887,6121,98624,93203,3,18057,24619,2,2,6970,1184,152,319,945,1102,1082
top,06/28/2019,06/28/2019,06/28/2019,07/01/2009,06/26/2008,11/12/2015,$0.00,$100.00,STANDARD,0,0,Y,Y,0,0,0,0,0,0,1
freq,1717,1700,1702,575,663,487,477545,144800,1600942,1704849,1672970,45815,32140,1713489,1677514,821444,396296,821462,454050,117779


In [93]:
# Need to convert date columns to pd.datetime
# RENAME PAID TO PAID DATE
# RENAME FULLY PAID TO FULLY PAID DATE
# RENAME ASSISGNED TO ASSIGNED DATE
# RENAME APPROVED TO APPROVED DATE

# CHECK COHERENCE OF PAID DATE <= FULLY PAID DATE
# CHECK COHERENCE OF PRE FILING DATE <= PAID DATE
# CHECK COHERENCE OF ASSIGNED DATE <= APPROVED DATE
# 

# REMOVE $ FROM Initial Cost and Total Est. Fee, and put them in column name, convert values to floats

# Check What fee status is

# Check Existing Zoning Sqft, Propsed Zoning Sqft, Enlargement SQ Footage for reasonable values
# Change either Sqft to SQ Footage or vvice-versa

# Horizontal Enlrgmt and Vertical Enlrgmt are booleans + NAN
# Change Enlrgmt to Enlargement

#Chcek ExisitngNo. of Stories and proposed No. of Stories for reasonableness
# Add space between Existing and No.
# Change either Job# to Job No. or vice versa
# maybe just change all the No./# to "number"

# Check Existing and proposed height for reasonableness. Add unit to column name

# check Existing Dwelling Units for reasonableness 



In [89]:
df[df.columns[60:80]].describe()

,Proposed Dwelling Units,Existing Occupancy,Proposed Occupancy,Site Fill,Zoning Dist1,Zoning Dist2,Zoning Dist3,Special District 1,Special District 2,Owner Type,Non-Profit,Owner's First Name,Owner's Last Name,Owner's Business Name,Owner's House Number,Owner'sHouse Street Name,City,State,Zip,Owner'sPhone #
count,918738,1270588,1062751,1424995,1440051,193632,10263,262130,79909,1701439,1703286,1774553,1774618,1424233,1003,1003,1003,1003,1001,1763366
unique,1173,47,49,5,229,196,135,116,15,14,2,93115,156866,412712,549,756,114,9,212,359264
top,1,RES,RES,NOT APPLICABLE,R6,R8B,C6-6,MID,POPS,CORPORATION,N,MICHAEL,SINGH,OWNER,2,555 WEST 23RD STREET,BROOKLYN,NY,11201,7184728000
freq,172057,411650,289996,717112,149653,19305,1554,84261,27848,575526,1594148,40957,23629,11053,19,17,309,981,46,12731


In [ ]:
# check Proposed Dwelling Units
## Why does Existing Occupancy have fewer cats than Proposed Occupancy. Check those.

# What is Site Fill. 

# Get list of NYC ZOning Districts and Special Districts

# Checmk Owner Typer for spelling issues

# Non Profit is binary

# Check Owners's First and last name

# Owner'sBuisness Name should not be "OWNER"

# Owner's house number, streeet name, city, state and zip have almost no values

# Why is the same phone number so common

In [90]:
df[df.columns[80:100]].describe()

,Job Description,DOBRunDate,JOB_S1_NO,TOTAL_CONSTRUCTION_FLOOR_AREA,WITHDRAWAL_FLAG,SIGNOFF_DATE,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
count,1594497,1774833,1774833,1774833,1774833,1063242,1754629,172179,1768356,1774833,1769176,1769176,1769176,1769176,1769176,1752710
unique,1247237,297,1774833,26598,13,5919,4,6373,199,13,217559,228333,51,1328,194,342789
top,PROPOSED INSTALLATION OF HEAVY DUTY SIDEWALK S...,04/28/2021 00:00:00,3293955,0,0,02/12/2018,N,10/24/2012,R0,0,40.748276,-73.984690,4,7,Midtown-Midtown South,3000000
freq,3985,1676399,1,1619529,1752630,603,1519403,5513,209462,1772918,2409,2413,236660,16439,153870,15627


In [94]:

# That is a lot of unique job descriptions

# Add spaces to DOBRunDate name
# make DOBRunDate a datetime

# What is Job_S1_NO. It uses underscores.

# All the remaining columns have ALL CAPS NAMES WITH UNDERSCORES 
# TOTAL_CONSTRUCTION_FLOOR_AREA, WITHDRAWAL_FLAG

# SIGNOFF_DATE needs to be datetime
# SPECIAL_ACTION_STATUS
# SPECIAL_ACTION_DATE needs to be datetime
# BUILDING_CLASS
# What is JOB_NO_GOOD_COUNT
#
# maybe need GIS DATA
# GIS_LATITUDE
# GIS_LONGITUDE
# GIS_COUNCIL_DISTRICT
# GIS_CENSUS_TRACT
# GIS_NTA_NAME
# GIS_BIN
# 

In [97]:
df.columns

Index(['Job #', 'Doc #', 'Borough', 'House #', 'Street Name', 'Block', 'Lot',
       'Bin #', 'Job Type', 'Job Status', 'Job Status Descrp',
       'Latest Action Date', 'Building Type', 'Community - Board', 'Cluster',
       'Landmarked', 'Adult Estab', 'Loft Board', 'City Owned', 'Little e',
       'PC Filed', 'eFiling Filed', 'Plumbing', 'Mechanical', 'Boiler',
       'Fuel Burning', 'Fuel Storage', 'Standpipe', 'Sprinkler', 'Fire Alarm',
       'Equipment', 'Fire Suppression', 'Curb Cut', 'Other',
       'Other Description', 'Applicant's First Name', 'Applicant's Last Name',
       'Applicant Professional Title', 'Applicant License #',
       'Professional Cert', 'Pre- Filing Date', 'Paid', 'Fully Paid',
       'Assigned', 'Approved', 'Fully Permitted', 'Initial Cost',
       'Total Est. Fee', 'Fee Status', 'Existing Zoning Sqft',
       'Proposed Zoning Sqft', 'Horizontal Enlrgmt', 'Vertical Enlrgmt',
       'Enlargement SQ Footage', 'Street Frontage', 'ExistingNo. of Stories',
  

In [159]:
def show_vals(column_name, show_rows=10, df=df):
    print("Top {} {}:\n".format(show_rows, column_name))
    print(df[column_name].value_counts(dropna=False)[:show_rows])
    print()

### House numbers can't be strings because some have dashes

In [160]:
show_vals('House #')

Top 10 House #:

1      17359
200    11853
100     8114
11      7231
10      7191
55      7166
40      7105
30      7098
2       6428
150     6380
Name: House #, dtype: int64



### House numbers shouldn't be words

In [161]:
df.loc[(~df['House #'].isna())
       &(df['House #'].str.isalpha())]['House #']

15545            ONE
76256            nan
76440            nan
76851            nan
76853            nan
76859            nan
76863            nan
1173934         PIER
1180684         PIER
1345596          ONE
1345764          ONE
1378164          ONE
1400111          ONE
1405705          ONE
1513683    MANHATTAN
Name: House #, dtype: object

In [162]:
df['House #'] = df['House #'].astype('str')

In [163]:
df.loc[(~df['House #'].isna())
       &(~df['House #'].str.isdigit())]['House #']

0           215-02
2            88-36
3            88-36
6            34-74
15           34-74
            ...   
1774620    186-200
1774621    186-200
1774647    372-380
1774670    633-639
1774815      26-28
Name: House #, Length: 332182, dtype: object

### These look like Apartment numbers mostly

In [164]:
df.loc[(~df['House #'].isna())
       &(~df['House #'].str.isdigit())
      &(~df['House #'].str.contains('-'))]['House #']

40           215B
73         270GAR
86         9 WEST
212          128A
274         1715A
            ...  
1773904        7A
1774070      486A
1774488     539 A
1774494      140B
1774572      277A
Name: House #, Length: 11050, dtype: object

## Looking at Binary columns:

In [165]:
show_vals('Landmarked')
show_vals('Adult Estab')
show_vals('Loft Board')
show_vals('City Owned')
show_vals('Little e')
show_vals('PC Filed')
show_vals('eFiling Filed')


Top 10 Landmarked:

N      1461463
Y       208859
NaN     104119
L          369
C           23
Name: Landmarked, dtype: int64

Top 10 Adult Estab:

N      1581220
NaN     193440
Y          173
Name: Adult Estab, dtype: int64

Top 10 Loft Board:

N      1393358
NaN     374039
Y         7436
Name: Loft Board, dtype: int64

Top 10 City Owned:

NaN    1625442
Y       149388
I            1
M            1
N            1
Name: City Owned, dtype: int64

Top 10 Little e:

N      989635
NaN    728416
Y       36055
H       20722
M           4
X           1
Name: Little e, dtype: int64

Top 10 PC Filed:

NaN    1172627
Y       384720
N       217486
Name: PC Filed, dtype: int64

Top 10 eFiling Filed:

Y      1081478
NaN     693353
R            2
Name: eFiling Filed, dtype: int64



In [166]:
show_vals('Plumbing')
show_vals('Mechanical')
show_vals('Boiler')
show_vals('Fuel Burning')
show_vals('Fuel Storage')
show_vals('Standpipe')
show_vals('Sprinkler')
show_vals('Fire Alarm')
show_vals('Equipment')
show_vals('Fire Suppression')
show_vals('Curb Cut')
show_vals('Other')

Top 10 Plumbing:

NaN    1219121
X       555712
Name: Plumbing, dtype: int64

Top 10 Mechanical:

NaN    1490536
X       284297
Name: Mechanical, dtype: int64

Top 10 Boiler:

NaN    1724873
X        49960
Name: Boiler, dtype: int64

Top 10 Fuel Burning:

NaN    1753381
X        21452
Name: Fuel Burning, dtype: int64

Top 10 Fuel Storage:

NaN    1760578
X        14254
N            1
Name: Fuel Storage, dtype: int64

Top 10 Standpipe:

NaN    1760447
X        14386
Name: Standpipe, dtype: int64

Top 10 Sprinkler:

NaN    1658447
X       116386
Name: Sprinkler, dtype: int64

Top 10 Fire Alarm:

NaN    1687130
X        87703
Name: Fire Alarm, dtype: int64

Top 10 Equipment:

NaN    1468021
X       306812
Name: Equipment, dtype: int64

Top 10 Fire Suppression:

NaN    1731463
X        43370
Name: Fire Suppression, dtype: int64

Top 10 Curb Cut:

NaN    1713620
X        61213
Name: Curb Cut, dtype: int64

Top 10 Other:

X      1041742
NaN     733091
Name: Other, dtype: int64



In [167]:
# Many of these values are the same but spelled differently
show_vals('Other Description')

Top 10 Other Description:

NaN                734893
GEN. CONSTR.       316603
GC                 116243
GEN. CONSTR         87351
STRUCTURAL          55576
BPP                 50685
GEN CONST           42153
SOLAR TAX           33561
GEN.CONSTRUCTN.     26544
FACADE              22633
Name: Other Description, dtype: int64



In [168]:
# Should remove "MR" when found
show_vals("Applicant's First Name")

Top 10 Applicant's First Name:

MICHAEL    51058
JOHN       44295
ROBERT     41756
DAVID      35926
PAUL       32698
ANTHONY    29881
JAMES      27059
JOSEPH     26991
THOMAS     24207
PETER      21916
Name: Applicant's First Name, dtype: int64



In [174]:
df["Applicant's First Name"] = df["Applicant's First Name"].astype('str')

In [175]:
# Unclear if these are actual first names mostly (Except for WU (woody))
df.loc[df["Applicant's First Name"].str.contains(" ")]["Applicant's First Name"].value_counts()

MD TOHFAZ      3090
KO SHEK        1633
JUNG WOR       1228
PING MAN       1133
WU (WOODY)     1119
               ... 
SARA MARSH        1
F. JOSPEH         1
JOAN BOOYE        1
JOHN OLIVER       1
SARAH MARSH       1
Name: Applicant's First Name, Length: 2790, dtype: int64

In [176]:
df["Applicant's Last Name"] = df["Applicant's Last Name"].astype('str')

In [177]:
# At least one of thesse is not all caps, and clearly there are periods in some
show_vals("Applicant's Last Name")

Top 10 Applicant's Last Name:

HOQUE         19159
KATZ          15126
LEE           14512
RUDIKOFF      12109
GERAZOUNIS    11126
BAILEY        10826
CALIENDO       9306
CHEN           9110
SYED-NAQVI     7299
MASS           6665
Name: Applicant's Last Name, dtype: int64



In [181]:
# Most of these are the same
show_vals("Applicant Professional Title", show_rows=50)

Top 50 Applicant Professional Title:

PE                 924243
RA                 796674
GC                  12762
OT                  10617
G.C                  5707
CONTRACTOR           3314
DEMO CONTRACTOR      2087
DEMO                 1973
PERMIT BROKER        1765
GEN CONTRACTOR       1504
G.C.                 1492
Contractor           1355
OWNER                1228
Demo Contractor       639
X                     534
DEMO CONTR            476
GEN. CONTRACTOR       451
SIGN HANGER           350
GENERAL CONTRAC       348
LA                    267
SIGN PERMIT AGT       248
Demolition Cont       242
SUBCONTRACTOR         232
Owner                 194
G. CONTRACTOR         177
GEN.CONTRACTOR        172
PLLC                  161
LAND SURVEYOR         131
G. C.                 129
General Contrac       124
GEN CONT              108
DEMO CONT             106
DEMO. CONTR           105
contractor            100
Gen. Contractor        95
G C                    94
G CONTRACTOR           93


In [184]:
show_vals('Applicant License #', show_rows=20)

Top 20 Applicant License #:

NaN       49167
083278    19195
050579    16342
066709    12092
013529    10886
082770    10711
020419    10230
020241     9350
080123     8246
038964     7522
045101     6732
058375     6614
076082     6570
066791     6334
035290     6106
053541     6031
054953     5945
029017     5627
075316     5354
071719     5290
Name: Applicant License #, dtype: int64



In [186]:
# Some of these are obviously wrong
show_vals('Professional Cert')

Top 10 Professional Cert:

Y      859575
N      537059
NaN    378195
J           2
F           1
R           1
Name: Professional Cert, dtype: int64

